## Importação

In [2]:
import pandas as pd
import zipfile
import os

zip_path = "datasets/renaest_dabertos_20250912.zip"
out_dir = "datasets/cleaned"

os.makedirs(out_dir, exist_ok=True)

## pré-processamento inicial e otimização

In [3]:
def optimize_df(df: pd.DataFrame) -> pd.DataFrame:
    """Otimiza dtypes de um DataFrame para reduzir uso de memória."""
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['object']).columns:
        nunique = df[col].nunique(dropna=False)
        if nunique < df.shape[0] * 0.5:  # baixa cardinalidade
            df[col] = df[col].astype('category')
    return df

In [4]:
def clean_missing_values(df: pd.DataFrame, dataset_name: str) -> pd.DataFrame:
    """Remove/trata valores não informados e desconhecidos."""
    df_clean = df.copy()
    
    missing_patterns = [
        'NAO INFORMADO', 'NÃO INFORMADO', 'DESCONHECIDO', 
        'SEM INFORMACAO', 'SEM INFORMAÇÃO', 'NULL', 'NAN',
        'Não informado', 'Desconhecido', 'IGNORADO', 'INDETERMINADO'
    ]
    
    if dataset_name == 'acidentes':
        valid_days = ['SEGUNDA-FEIRA', 'TERCA-FEIRA', 'QUARTA-FEIRA', 
                      'QUINTA-FEIRA', 'SEXTA-FEIRA', 'SABADO', 'DOMINGO']
        if 'dia_semana' in df_clean.columns:
            df_clean = df_clean[df_clean['dia_semana'].isin(valid_days)]
        
        if 'cond_pista' in df_clean.columns:
            mask_pista_valida = ~df_clean['cond_pista'].astype(str).str.upper().isin(missing_patterns)
            df_clean = df_clean[mask_pista_valida]
            
            def clean_road_condition(value):
                if pd.isna(value):
                    return 'OUTROS'
                value_str = str(value).upper()
                if 'MOLHADA' in value_str or 'ESCORREGADIA' in value_str:
                    return 'MOLHADA'
                elif 'SECA' in value_str:
                    return 'SECA'
                else:
                    return 'OUTROS'
            
            df_clean['cond_pista_simples'] = df_clean['cond_pista'].apply(clean_road_condition)
        
        if 'cond_meteorologica' in df_clean.columns:
            mask_clima_valido = ~df_clean['cond_meteorologica'].astype(str).str.upper().isin(missing_patterns)
            df_clean = df_clean[mask_clima_valido]
        
        if 'dia_semana' in df_clean.columns:
            df_clean['fim_semana'] = df_clean['dia_semana'].isin(['SABADO', 'DOMINGO'])
    
    elif dataset_name == 'veiculos':
        main_vehicles = ['AUTOMOVEL', 'MOTOCICLETA', 'CAMINHAO', 'ONIBUS', 'BICICLETA']
        if 'tipo_veiculo' in df_clean.columns:
            df_clean = df_clean[df_clean['tipo_veiculo'].isin(main_vehicles)]
            mask_veiculo_valido = ~df_clean['tipo_veiculo'].astype(str).str.upper().isin(missing_patterns)
            df_clean = df_clean[mask_veiculo_valido]
    
    elif dataset_name == 'vitimas':
        if 'faixa_idade' in df_clean.columns:
            mask = ~df_clean['faixa_idade'].astype(str).str.upper().isin(missing_patterns)
            df_clean = df_clean[mask]
            
            def clean_age_category(age_range):
                if pd.isna(age_range):
                    return None  
                age_str = str(age_range).upper()
                
                if any(pattern in age_str for pattern in missing_patterns):
                    return None
                
                if any(y in age_str for y in ['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34']):
                    return 'JOVEM (18-34)'
                elif any(y in age_str for y in ['35', '40', '45', '50', '55', '59']):
                    return 'ADULTO (35-59)'
                elif '60' in age_str or 'MAIOR' in age_str or 'MAIS' in age_str:
                    return 'IDOSO (60+)'
                else:
                    return 'OUTROS'
            
            df_clean['faixa_idade_simples'] = df_clean['faixa_idade'].apply(clean_age_category)
            df_clean = df_clean[df_clean['faixa_idade_simples'].notna()]
    
    return df_clean

## Importar e salvar

In [5]:
with zipfile.ZipFile(zip_path, "r") as z:
    for filename in z.namelist():
        if not filename.endswith(".csv"):
            continue
        print(f"📂 Lendo {filename} ...")
        with z.open(filename) as f:
            df = pd.read_csv(f, sep=";", low_memory=False)
            df = optimize_df(df)
            
            dataset_name = None
            if 'Acidentes' in filename:
                dataset_name = 'acidentes'
            elif 'TipoVeiculo' in filename:
                dataset_name = 'veiculos'
            elif 'Vitimas' in filename:
                dataset_name = 'vitimas'
            elif 'Localidade' in filename:
                dataset_name = 'localidades'
            
            if dataset_name:
                df = clean_missing_values(df, dataset_name)
                print(f"   🧹 Limpeza aplicada para {dataset_name}")

            base = filename.replace(".csv", "")
            out_file = f"{out_dir}/{base}.parquet"
            df.to_parquet(out_file, index=False, engine="pyarrow")

            print(f"✅ {filename} salvo como {out_file} ({df.shape[0]} linhas, {df.shape[1]} colunas)")

print("\n🚀 Importação e limpeza concluídas! Arquivos limpos em:", out_dir)

📂 Lendo Acidentes_DadosAbertos_20250912.csv ...
   🧹 Limpeza aplicada para acidentes
✅ Acidentes_DadosAbertos_20250912.csv salvo como datasets/cleaned/Acidentes_DadosAbertos_20250912.parquet (1673564 linhas, 37 colunas)
📂 Lendo Localidade_DadosAbertos_20250912.csv ...
   🧹 Limpeza aplicada para localidades
✅ Localidade_DadosAbertos_20250912.csv salvo como datasets/cleaned/Localidade_DadosAbertos_20250912.parquet (498133 linhas, 12 colunas)
📂 Lendo TipoVeiculo_DadosAbertos_20250912.csv ...
   🧹 Limpeza aplicada para veiculos
✅ TipoVeiculo_DadosAbertos_20250912.csv salvo como datasets/cleaned/TipoVeiculo_DadosAbertos_20250912.parquet (6860268 linhas, 4 colunas)
📂 Lendo Vitimas_DadosAbertos_20250912.csv ...
   🧹 Limpeza aplicada para vitimas
✅ Vitimas_DadosAbertos_20250912.csv salvo como datasets/cleaned/Vitimas_DadosAbertos_20250912.parquet (8028049 linhas, 18 colunas)

🚀 Importação e limpeza concluídas! Arquivos limpos em: datasets/cleaned
